**Домашняя работа по Алгоритмам в Биоинформатике №10**



---



1. Построение графа Де Брюина (5 баллов)
По заданному набору ридов в формате FASTQ и параметру k, который соответствует длине k-меров, построить граф Де Брюина, некоторый путь в котором соответствовал бы возможной подстроке в исходном геноме. Не забывайте про запоминание покрытия каждого k-мера, а так же про сами подстроки, которые соответствуют каждому ребру. В остальном граф полностью соответствует тому, что был описан в лекции.

In [ ]:
!apt install libcairo2-dev
!pip install pycairo


In [ ]:
!pip install git+https://github.com/sandialabs/toyplot

In [ ]:
import random
import toyplot

In [ ]:
class KmerAnalyzer:
    def __init__(self, k=5, cyclic=True):
        self.k = k
        self.cyclic = cyclic

    def get_kmers_from_sequence(self, sequence):
        """
        Returns list with all possible kmers in a sequence
        """
        kmers = []
        for i in range(0, len(sequence)):
            kmer = sequence[i:i + self.k]
            length = len(kmer)
            if self.cyclic:
                if length != self.k:
                    kmer += sequence[:(self.k - length)]
            else:
                if length != self.k:
                    continue
            kmers.append(kmer)
        return kmers

    def get_kmers_from_fastq_file(self, fastq_file_path):
        """
        Returns dictionary with keys representing all possible kmers in a FASTQ file
        and values counting their occurrence in the file.
        """
        kmers = {}
        with open(fastq_file_path, "r") as fastq_file:
            for line in fastq_file:
                if line.startswith("@"):
                    sequence = next(fastq_file).strip()
                    kmers_in_sequence = self.get_kmers_from_sequence(sequence)
                    for kmer in kmers_in_sequence:
                        if kmer in kmers:
                            kmers[kmer] += 1
                        else:
                            kmers[kmer] = 1
                    next(fastq_file)
                    next(fastq_file)
        return kmers

    def get_debruijn_edges_from_kmers(self, kmers):
        """
        Every possible (k-1)mer (n-1 suffix and prefix of kmers) is assigned
        to a node, and we connect one node to another if the (k-1)mer overlaps 
        another. Nodes are (k-1)mers, edges are kmers.
        """
        edges = set()
        for k1 in kmers:
            for k2 in kmers:
                if k1 != k2:            
                    if k1[1:] == k2[:-1]:
                        edges.add((k1[:-1], k2[:-1]))
                    if k1[:-1] == k2[1:]:
                        edges.add((k2[:-1], k1[:-1]))
        return edges


    def plot_debruijn_graph(self, edges, width=1000, height=1000):
        graph = toyplot.graph(
            [i[0] for i in edges],
            [i[1] for i in edges],
            width=width,
            height=height,
            tmarker=">", 
            vsize=40,
            vstyle={"stroke": "black", "stroke-width": 2, "fill": "none"},
            vlstyle={"font-size": "14px"},
            estyle={"stroke": "black", "stroke-width": 2},
            layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
        return graph

    def remove_bad_coverage_tails(self, edges, kmer_counts):
        """
        Removes edges from the De Bruijn graph that correspond to tails with low coverage.
        """
        # Get all tails
        tails = set()
        for kmer in kmer_counts:
            tails.add(kmer[-(self.k - 1):])
        
        # Calculate distribution of coverage multiplied by tail length
        tail_coverages = []
        for tail in tails:
            tail_coverage = 0
            for kmer, count in kmer_counts.items():
                if kmer.endswith(tail):
                    tail_coverage += count
            tail_coverages.append(tail_coverage * len(tail))
        tail_coverages = np.array(tail_coverages)
        
        # Identify tails with low coverage
        threshold = np.percentile(tail_coverages, 30)
        bad_tails = set()
        for tail in tails:
            tail_coverage = 0
            for kmer, count in kmer_counts.items():
                if kmer.endswith(tail):
                    tail_coverage += count
            if tail_coverage * len(tail) <= threshold:
                bad_tails.add(tail)
        
        # Remove edges corresponding to bad tails
        cleaned_edges = []
        for edge in edges:
            if edge[0][-1] not in bad_tails and edge[1][-1] not in bad_tails:
                cleaned_edges.append(edge)
        return cleaned_edges


In [ ]:
kmer_analyzer = KmerAnalyzer(k=4)
kmers = kmer_analyzer.get_kmers_from_fastq_file("example.fastq")
debruijn_edges = kmer_analyzer.get_debruijn_edges_from_kmers(kmers)
kmer_analyzer.plot_debruijn_graph(debruijn_edges)


(<toyplot.canvas.Canvas at 0x7f2188b1e530>,
 <toyplot.mark.Graph at 0x7f2188b1ead0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t45aef4ec75fb4fafb7acd2854bc65570"> AAA AAC AAG ACA ACT AGA AGC AGG AGT ATC ATT CAG CAT CCG CGA CGC CGG CGT CTA CTC GAT GCA GCC GCG GCT GGC GGG GGT GTG GTT TAA TAC TAG TCA TCG TGC TGG TTA TTG TTT

2. Сжатие графа (4 балла)
Научитесь производить сжатие графа Де Брюина. При сжатии не забывайте склеивать подстроки на ребрах и обновлять покрытие ребер. Пересчитывайте покрытие склеиваемых ребер как взвешенное среднее, где вес соответствует длине подстроки, соответствующей ребру


In [ ]:
def compress_graph(kmer_analyzer: KmerAnalyzer, edges):
    compressed_edges = set()
    coverage = {}
    for edge in edges:
        if edge in compressed_edges:
            continue
        node1, node2 = edge
        node1_coverage = coverage.get(node1, 0)
        node2_coverage = coverage.get(node2, 0)
        total_coverage = node1_coverage + node2_coverage
        total_length = len(node1) + len(node2) - (kmer_analyzer.k - 1)
        if total_coverage == 0:
            compressed_edges.add(edge)
            coverage[node1] = coverage[node2] = 0
        else:
            avg_coverage = total_coverage / total_length
            compressed_node = node1 + node2[-1]
            compressed_edges.add((node1, compressed_node))
            compressed_edges.add((compressed_node, node2))
            coverage[node1] = coverage[node2] = avg_coverage
            coverage[compressed_node] = avg_coverage
    return compressed_edges



In [ ]:
analyzer = KmerAnalyzer(k=4, cyclic=True)
kmers = analyzer.get_kmers_from_fastq_file("example.fastq")
edges = analyzer.get_debruijn_edges_from_kmers(kmers)
compressed_edges = compress_graph(analyzer, edges)
compressed_graph = analyzer.plot_debruijn_graph(compressed_edges)



<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tabe9390c46d4413489efe8ef22258f86"> AAA AAC AAG ACA ACT AGA AGC AGG AGT ATC ATT CAG CAT CCG CGA CGC CGG CGT CTA CTC GAT GCA GCC GCG GCT GGC GGG GGT GTG GTT TAA TAC TAG TCA TCG TGC TGG TTA TTG TTT

3. Удаление хвостов (3 балла)
Реализуйте возможность после построения графа удалять плохо покрытые хвосты. Чтобы определить, является ли хвост плохо покрытым, нужно посчитать распределение произведений покрытий на длину хвоста по всем хвостам и удалить те, что попадают в 30 процентов ближе к нулю распределения.

In [ ]:
# Создаем экземпляр класса
analyzer = KmerAnalyzer(k=4, cyclic=True)

# Получаем k-меры из FASTQ файла
kmer_counts = analyzer.get_kmers_from_fastq_file("example.fastq")

# Получаем ребра графа De Bruijn
edges = analyzer.get_debruijn_edges_from_kmers(kmer_counts)

# Удаляем плохо покрытые хвосты
cleaned_edges = analyzer.remove_bad_coverage_tails(edges, kmer_counts)

# Визуализируем граф
analyzer.plot_debruijn_graph(cleaned_edges)


(<toyplot.canvas.Canvas at 0x7f2188900250>,
 <toyplot.mark.Graph at 0x7f2188903640>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tad9e8dc5e72947628cacf0de84384c79"> AAA AAC AAG ACA ACT AGA AGC AGG AGT ATC ATT CAG CAT CCG CGA CGC CGG CGT CTA CTC GAT GCA GCC GCG GCT GGC GGG GGT GTG GTT TAA TAC TAG TCA TCG TGC TGG TTA TTG TTT

4. Удаление пузырей (4 балла)
Научитесь удалять такие пузыри в графе, длина которых меньше либо равна 2k. Таким образом должно получится удаление последствий единичных ошибок в ридах.


Здесь используется следующая идея: пузырь - это две ветви, которые начинаются и заканчиваются в одном и том же узле, и между которыми есть еще одна ветвь, соединяющая их вместе. Последний элемент первой ветви должен совпадать с первым элементом второй ветви. Мы можем найти все такие узлы с помощью обхода графа и проверки, имеют ли они ровно два входящих и исходящих ребра. Затем мы можем удалить пузырь, если его длина меньше или равна 2k.

In [ ]:
def remove_bubbles(edges, k=4):
    """
    Remove bubbles in the graph whose length is less than or equal to 2k.
    """
    nodes = set()
    for edge in edges:
        nodes.update(edge)

    # Find all the nodes with degree 2.
    degree_2_nodes = set()
    for node in nodes:
        incoming_edges = set(filter(lambda e: e[1] == node, edges))
        outgoing_edges = set(filter(lambda e: e[0] == node, edges))
        if len(incoming_edges) == 1 and len(outgoing_edges) == 1:
            degree_2_nodes.add(node)

    # Remove the bubble edges.
    bubble_edges = set()
    for node in degree_2_nodes:
        incoming_edge = next(filter(lambda e: e[1] == node, edges))
        outgoing_edge = next(filter(lambda e: e[0] == node, edges))
        bubble = incoming_edge[0] + outgoing_edge[1]
        if len(bubble) <= 2*k:
            bubble_edges.update((incoming_edge, outgoing_edge))

    return edges - bubble_edges


In [ ]:
analyzer = KmerAnalyzer(k=4)
kmers = analyzer.get_kmers_from_fastq_file('example.fastq')
edges = analyzer.get_debruijn_edges_from_kmers(kmers)
edges = remove_bubbles(edges, k=4)
analyzer.plot_debruijn_graph(edges)



(<toyplot.canvas.Canvas at 0x7f2188bb83a0>,
 <toyplot.mark.Graph at 0x7f2188bbb1c0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="1000.0px" viewBox="0 0 1000.0 1000.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taf9ce114c06d41a0885329d5c4c1db9c"> AAA AAC ACT ATT CGC CTA GAT GCG GCT GGC GGT GTT TAA TGC TGG TTA TTG TTT

Самым эффективным методом на примере тестовых данных оказалася метод удаления пузырей, он привел к значительному упрощению графа. Остальные методы не дали значительного результата в ввиду того, что на тестовых данных отсутствуют данные артефакты.